In [1]:
import json
f = open('./target_seq.txt')
target_seqs = json.load(f)
f.close()
g= open('./SMILES.txt')
smiles = json.load(g)
g.close()

In [2]:
from Bio.PDB.PDBParser import PDBParser
from Bio.PDB.Polypeptide import three_to_one
from Bio.PDB.Polypeptide import is_aa
from Bio import pairwise2
from multiprocessing import Pool, cpu_count
from functools import partial
import scipy.cluster.hierarchy
import numpy as np
import sys
import argparse
import bisect
import re
import os
import fnmatch
import pickle
import collections
from rdkit.Chem import AllChem as Chem
from rdkit.Chem import AllChem
from rdkit.DataStructs import FingerprintSimilarity as fs
from rdkit.Chem.Fingerprints import FingerprintMols
import rdkit

In [3]:
def calcDistanceMatrix(targets):
    '''compute full pairwise target distance matrix in parallel'''
    n = len(targets)
    pairs = [(r, c) for r in range(n) for c in range(r+1, n)]  # upper triangle
#     print("list of pairs", pairs)

    pool = Pool()
    function = partial(cUTDM2, targets)
    distanceTuples = pool.map(function, pairs)
    distanceMatrix = np.zeros((n, n))
    for (a, b, distance) in distanceTuples:
        distanceMatrix[a][b] = distanceMatrix[b][a] = distance
    
    print("dist mat done")
    return distanceMatrix


In [4]:
def cUTDM2(targets, pair):
    '''compute distance between target pair'''
    (a, b) = pair
#     print("pair", pair)

    mindist = 1.0
    for seq1 in targets[a]:
        for seq2 in targets[b]:
            score = pairwise2.align.globalxx(seq1, seq2, score_only=True)
            length = max(len(seq1), len(seq2))
            distance = (length-score)/length
            if distance < mindist:
                mindist = distance
#     print(a, b, mindist)
    return (a, b, mindist)

In [5]:
def get_targets_only():
    targets = []
    target_names= []
    for t in target_seqs:
        target_names.append(t)
        targets.append(target_seqs[t])
    targets = np.reshape(targets, (len(targets), 1))
    return target_names, targets

In [6]:
target_names, targets = get_targets_only()

# Making distance matrix

In [9]:
distanceMatrix = calcDistanceMatrix(targets)

dist mat done


In [10]:
np.shape(distanceMatrix)

(229, 229)

In [11]:
def newcomputeLigandSimilarity():
    
    fingerprints = dict()
    for smile in smiles:
        drug_name =smile
        smi = smiles[smile]
        mol = AllChem.MolFromSmiles(smi)
        if mol == None:
            mol = AllChem.MolFromSmiles(smi, sanitize=False)
        fp = FingerprintMols.FingerprintMol(mol)
        fingerprints[smile] = fp
    n = len(smiles)
    drug_names = list(smiles.keys())
    sims = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1):
            fpi = fingerprints[drug_names[i]]
            fpj = fingerprints[drug_names[j]]
            sim = fs(fpi, fpj)
            sims[i, j] = sims[j, i] = sim
    return sims, fingerprints

In [12]:
similarityMatrix, fp= newcomputeLigandSimilarity()

In [13]:
ligandsim = similarityMatrix
np.shape(ligandsim)

(2111, 2111)

In [14]:
def get_target_lines(file):
    file = open(file, 'r')
    lines = file.readlines()
    ret = collections.defaultdict(list)
    
    for line in lines:
        x = line.split()
        targ = x[0]
        ret[targ].append(line)

    return ret

In [15]:
target_lines = get_target_lines('og_kiba_fname.txt')

In [16]:
all_pairs=[]
f = open('./og_kiba_fname.txt')
lines = f.readlines()
c = 0
for line in lines:
    l = line.split()
    all_pairs.append([c,l[0],l[1]])
    c= c+1

In [17]:
similarity = 0.5
similarity_with_similar_ligand = 0.4
ligand_similarity = 0.9
threshold = 1 - similarity  # similarity and distance are complementary
threshold2 = 1 - similarity_with_similar_ligand
ligand_threshold =ligand_similarity

In [21]:

def calcClusterGroups(dists, ligandsim, target_names, t, t2, ligandt, all_pairs):
    '''dists is a distance matrix (full) for target_names'''
    assigned = set()
    groups = []
    for i in range(dists.shape[0]):
        
        if i not in assigned:
            group = assignGroup(dists, ligandsim, t, t2,
                                ligandt, set([i]), target_names)
            groups.append(group)
            assigned.update(group)
    return [set(target_names[i] for i in g) for g in groups]


def assignGroup(dists, ligandsim, t, t2, ligandt, explore, names):
    '''group targets that are less than t away from each other and what's in explore'''
    group = set(explore)
    while explore:
        frontier = set()
        for i in explore:
            for j in range(dists.shape[1]):
                if j not in group:
                    # add to the group if protein is close by threshold t (these are distances - default 0.5)
                    # also add if the ligands are more similar (not distance) than ligandt and
                    # the protein is closer than t2 (default 0.8 - meaning more than 20% similar)
                    if dists[i][j] < t: # or (ligandsim[i][j] > ligandt and dists[i][j] < t2):
                        group.add(j)
                        frontier.add(j)

        explore = frontier
    return group

In [22]:
cluster_groups = calcClusterGroups(
            distanceMatrix, ligandsim, target_names, threshold, threshold2, ligand_threshold, all_pairs)

In [23]:
cluster_groups

[{'O00141', 'Q9HBY8'},
 {'O00311'},
 {'O00329', 'P42338'},
 {'O00418'},
 {'O00444'},
 {'O14757'},
 {'O14920', 'O15111'},
 {'O14965', 'Q96GD4'},
 {'O15075'},
 {'O15264', 'P27361', 'P28482', 'P53778', 'Q15759', 'Q16539'},
 {'O15530'},
 {'O43293'},
 {'O43741', 'Q9Y478'},
 {'O43781'},
 {'O60285'},
 {'O60674', 'P23458', 'P29597', 'P52333'},
 {'O75116', 'Q13464'},
 {'O75582', 'O75676'},
 {'O94806', 'Q15139', 'Q9BZL6'},
 {'O95819', 'Q8N4C8'},
 {'O96013'},
 {'O96017'},
 {'P00519', 'P42684'},
 {'P00533', 'P04626', 'Q15303'},
 {'P04049', 'P15056'},
 {'P04629', 'Q16288', 'Q16620'},
 {'P05129', 'P05771', 'P17252', 'P24723', 'Q02156', 'Q04759', 'Q05655'},
 {'P06213', 'P08069'},
 {'P06239',
  'P06241',
  'P07947',
  'P07948',
  'P08631',
  'P09769',
  'P12931',
  'P42685',
  'P51451'},
 {'P06493', 'P11802', 'P24941', 'Q00534', 'Q00535'},
 {'P07332', 'P16591'},
 {'P07333', 'P10721'},
 {'P07949'},
 {'P08581'},
 {'P08922'},
 {'P09619', 'P16234'},
 {'P11309', 'Q86V86', 'Q9P1W9'},
 {'P11362', 'P21802', '

In [24]:
len(cluster_groups)

128

In [29]:
all_pairs_dict ={}
for t in target_seqs:
    all_pairs_dict.update({t:[]})
for i in all_pairs:
    t_n=i[1]
    smi =i[2]
    all_pairs_dict[t_n].append(smi)

In [30]:
# all_pairs_dict

In [33]:
a,b = cluster_groups[1], cluster_groups[0]
a,b

({'O00311'}, {'O00141', 'Q9HBY8'})

In [34]:
try_merge(a,b)

False

In [26]:
target_seq_index ={}
c = 0
for t in target_seqs:
    target_seq_index.update({t:c})
    c = c +1
smiles_Seq_index={}
smiles_list= list(smiles.values())
for i in smiles:
    smiles_Seq_index.update({smiles[i]:smiles_list.index(smiles[i])})

In [35]:
def try_merge(set1, set2):
    global distanceMatrix
    global all_pairs_dict
    global ligandsim
    new_set=set()
    for t1 in set1:
        for t2 in set2:
            t1_i = target_seq_index[t1]
            t2_i = target_seq_index[t2]
            
            if distanceMatrix[t1_i][t2_i] < 0.6:
                for lig1 in all_pairs_dict[t1]:
                    for lig2 in all_pairs_dict[t2]: 

                        l1_i=smiles_Seq_index[lig1]
                        l2_i =smiles_Seq_index[lig2]

                        if ligandsim[l1_i][l2_i]> 0.9:
                            new_set.update(set1)
                            new_set.update(set2)

                            return new_set
    return False



In [36]:
cluster_index={}
c=0
for i in cluster_groups:
    cluster_index.update({c:i})
    c= c +1
    
cluster_mat = np.zeros((len(cluster_groups), len(cluster_groups)))
for i in range(len(cluster_groups)):
    for j in range(len(cluster_groups)):
        if i != j:
            if try_merge(cluster_groups[i],cluster_groups[j]):
                cluster_mat[i][j] = cluster_mat[j][i] = 1

In [37]:
cluster_mat

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [38]:
cluster_hash=np.zeros(len(cluster_groups))
final_cluster_grps = []

assigned = set() # should contain the indices of the assigned clusters
groups = []
for i in range(len(cluster_groups)):
    cluster = set(cluster_groups[i])
    if i not in assigned:
        explore = set([i])
        group = set(explore)
        while explore:
            frontier = set()
            for i in explore:
                for j in range(len(cluster_groups)):
                    if j not in group:
                        boolean = try_merge(cluster_groups[i], cluster_groups[j])
                        if boolean:
#                             print(boolean)
                            group.add(j)
                            frontier.add(j)
            explore = frontier
        
        groups.append(group)
        assigned.update(group)

# final_clusters = [set(cluster_groups[i] for i in g) for g in groups]


In [44]:
# groups[0]

In [46]:
final_clusters=[]
for grp in groups:
    new_set = set()
    for i in grp:
        new_set.update(cluster_groups[i])
    final_clusters.append(new_set)

In [50]:
l=0
for i in final_clusters:
    l = l + len(i)
l

229

In [49]:
# final_clusters

In [51]:
def createFolds(cluster_groups, numfolds, target_lines, randomize):
    '''split target clusters into numfolds folds with balanced num poses per fold
       If randomize, will balance less well.
    '''
    folds = [[] for _ in range(numfolds)]
    fold_numposes = [0]*numfolds
    group_numposes = [0]*len(cluster_groups)
    foldmap = {}
    for i, group in enumerate(cluster_groups):
        # count num poses per group
        for target in group:
            group_numposes[i] += len(target_lines[target])
    for _ in cluster_groups:
        # iteratively assign group with most poses to fold with fewest poses
        maxgroup = group_numposes.index(np.max(group_numposes))
        if randomize:
            space = np.max(fold_numposes) - np.array(fold_numposes)
            tot = np.sum(space)
            if tot == 0:
                minfold = np.random.choice(numfolds)
            else:  # weighted selection, prefer spots with more free space
                choice = np.random.choice(tot)
                tot = 0
                for i in range(len(space)):
                    tot += space[i]
                    if choice < tot:
                        minfold = i
                        break
        else:
            minfold = fold_numposes.index(np.min(fold_numposes))
        folds[minfold].extend(cluster_groups[maxgroup])
        fold_numposes[minfold] += group_numposes[maxgroup]
        group_numposes[maxgroup] = -1
        for t in cluster_groups[maxgroup]:
            foldmap[t] = minfold
    print('Poses per fold: {}'.format(fold_numposes))
    for f in folds:
        f.sort()
    return folds, foldmap

def index(a, x):
    'Locate the leftmost value exactly equal to x'
    i = bisect.bisect_left(a, x)
    if i != len(a) and a[i] == x:
        return i
    else:
        return -1

In [52]:
folds, foldmap = createFolds(
            final_clusters, 3, target_lines, 0)

Poses per fold: [50137, 34065, 34052]


In [53]:
def crossvalidatefiles(folds, outname, numfolds, target_lines, reduce):
    # create test/train files
    trainfiles = [(open('{}train{}.types'.format(outname, i), 'w'), open(
        '{}reducedtrain{}.types'.format(outname, i), 'w')) for i in range(numfolds)]
    testfiles = [(open('{}test{}.types'.format(outname, i), 'w'), open(
        '{}reducedtest{}.types'.format(outname, i), 'w')) for i in range(numfolds)]
    target_set = set(sum(folds, []))

    for target in target_lines.keys():
        for i in range(numfolds):
            if target in folds[i]:
                out = testfiles[i]
            else:
                out = trainfiles[i]
            for line in target_lines[target]:
                out[0].write(line)
                if np.random.random() < reduce:
                    out[1].write(line)



In [54]:
crossvalidatefiles(folds, 'output', 3,
                           target_lines, 0.05)

In [55]:
folds

[['O00141',
  'O14757',
  'O14965',
  'O15075',
  'O15264',
  'O43293',
  'O60285',
  'O75582',
  'O75676',
  'P05129',
  'P05771',
  'P06493',
  'P11309',
  'P11802',
  'P15735',
  'P17252',
  'P17612',
  'P19784',
  'P22612',
  'P22694',
  'P23443',
  'P24723',
  'P24941',
  'P27361',
  'P27448',
  'P28482',
  'P31749',
  'P31751',
  'P34947',
  'P36507',
  'P41743',
  'P45983',
  'P45984',
  'P49137',
  'P49336',
  'P49840',
  'P49841',
  'P50613',
  'P50750',
  'P51812',
  'P51817',
  'P51955',
  'P52564',
  'P53350',
  'P53778',
  'P53779',
  'P54646',
  'P68400',
  'Q00534',
  'Q00535',
  'Q02156',
  'Q02750',
  'Q04759',
  'Q05513',
  'Q05655',
  'Q13131',
  'Q13237',
  'Q13554',
  'Q13555',
  'Q13557',
  'Q13976',
  'Q14012',
  'Q14680',
  'Q15418',
  'Q15759',
  'Q16539',
  'Q16566',
  'Q16644',
  'Q7KZI7',
  'Q86V86',
  'Q8IU85',
  'Q8IW41',
  'Q8TDC3',
  'Q96GD4',
  'Q96L34',
  'Q96PF2',
  'Q9BUB5',
  'Q9BWU1',
  'Q9BXA7',
  'Q9H0K1',
  'Q9H4B4',
  'Q9HBH9',
  'Q9HBY8',
  'Q